In [1]:
import pandas as pd
import os
import glob
import numpy as np
pd.options.mode.copy_on_write=True

# 2023

- Primero cargamos el dataset de acccidentalidad de Madrid del año 2023:

In [2]:
df_2023 = pd.read_csv(
    "./data/2023_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)

Ahora tambien quiero ver sus dimensiones, es decir cuantas filas y columnas tiene el Dataset

In [3]:
df_2023.shape

(48830, 19)

- Aquí observamos que tiene 19 columnas igual que 2025 y 2024, y tiene 48830 filas

Ahora para el formato real de los datos y detectar valores a simple vista

In [4]:
df_2023

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Conductor,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,443397.0,4478129.0,N,NaN
1,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 21 a 24 años,Hombre,14.0,Sin asistencia sanitaria,443397.0,4478129.0,N,NaN
2,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 21 a 24 años,Hombre,14.0,Sin asistencia sanitaria,443397.0,4478129.0,N,NaN
3,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 21 a 24 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.0,4478129.0,N,NaN
4,2023S000001,01/01/2023,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 55 a 59 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.0,4478129.0,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48825,2023S040267,31/12/2023,21:15:00,AVDA. GRAN VIA DE VILLAVERDE / AVDA. ANDALUCIA,10,17.0,VILLAVERDE,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 45 a 49 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,441152.0,4466350.0,N,NaN
48826,2023S040267,31/12/2023,21:15:00,AVDA. GRAN VIA DE VILLAVERDE / AVDA. ANDALUCIA,10,17.0,VILLAVERDE,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 6 a 9 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,441152.0,4466350.0,N,NaN
48827,2023S040277,29/12/2023,9:35:00,"PTA. TOLEDO, 0",0,1.0,CENTRO,Alcance,Despejado,Motocicleta hasta 125cc,Conductor,De 45 a 49 años,Hombre,NaN,NaN,439594.0,4473163.0,N,NaN
48828,2023S040277,29/12/2023,9:35:00,"PTA. TOLEDO, 0",0,1.0,CENTRO,Alcance,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,439594.0,4473163.0,N,NaN


In [5]:
df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48830 entries, 0 to 48829
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   num_expediente        48830 non-null  object 
 1   fecha                 48830 non-null  object 
 2   hora                  48830 non-null  object 
 3   localizacion          48830 non-null  object 
 4   numero                48827 non-null  object 
 5   cod_distrito          48827 non-null  float64
 6   distrito              48827 non-null  object 
 7   tipo_accidente        48830 non-null  object 
 8   estado_meteorológico  43127 non-null  object 
 9   tipo_vehiculo         48452 non-null  object 
 10  tipo_persona          48830 non-null  object 
 11  rango_edad            48830 non-null  object 
 12  sexo                  48830 non-null  object 
 13  cod_lesividad         25945 non-null  float64
 14  lesividad             25945 non-null  object 
 15  coordenada_x_utm   

Aquí procedemos a hacer una copia del dataset para dejar el original tal cual y poder ver diferencias a la hora de limpiar y demás.

In [6]:
df_2023_copy = df_2023.copy()

Vamos a generar una tabla para ver los nulos y la cardinalidad de las diferentes variables que nos ayudaran a tomar decisiones con cada variable.

In [7]:
# Número de nulos 
nulos = df_2023_copy.isna().sum() 

# Porcentaje de nulos 
nulos_pct = (nulos / len(df_2023_copy) * 100).round(2) 

# Cardinalidad 
card = df_2023_copy.nunique() 

# Porcentaje de cardinalidad 
card_pct = (card / len(df_2023_copy) * 100).round(2) 

# Tipo de dato 
tipos = df_2023_copy.dtypes 

#Construcción de la tabla 
tabla_eda = pd.DataFrame({ 
    "nulos": nulos,
    "% nulos": nulos_pct,
    "cardinalidad": card, 
    "% cardinalidad": card_pct,
    "tipo": tipos }) 

# Ordenar por cardinalidad (o por nulos si prefieres) 
tabla_eda = tabla_eda.sort_values("cardinalidad", ascending=False) 
tabla_eda

,nulos,% nulos,cardinalidad,% cardinalidad,tipo
num_expediente,0,0.00,20672,42.33,object
localizacion,0,0.00,15924,32.61,object
coordenada_y_utm,9,0.02,11071,22.67,float64
coordenada_x_utm,9,0.02,10170,20.83,float64
numero,3,0.01,1717,3.52,object
hora,0,0.00,1250,2.56,object
fecha,0,0.00,365,0.75,object
tipo_vehiculo,378,0.77,34,0.07,object
cod_distrito,3,0.01,21,0.04,float64
distrito,3,0.01,21,0.04,object


In [8]:
df_2023_copy["fecha"] = pd.to_datetime(df_2023_copy["fecha"], errors="coerce", dayfirst=True) 
df_2023_copy["hora"] = pd.to_datetime(df_2023_copy["hora"], format="%H:%M:%S", errors="coerce").dt.time 
df_2023_copy["fecha_hora"] = df_2023_copy.apply( 
                            lambda row: pd.Timestamp.combine(row["fecha"], row["hora"])
                            if pd.notnull(row["fecha"]) and pd.notnull(row["hora"])
                            else pd.NaT,
                            axis=1 )

cols_string = ["num_expediente", "localizacion", "numero"] 
for col in cols_string: 
    df_2023_copy[col] = df_2023_copy[col].astype("string").str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

- Ahora vamos a tratar los nulos 

In [9]:
# 1. ESTADO METEOROLÓGICO
df_2023_copy['estado_meteorológico'] = df_2023_copy['estado_meteorológico'].fillna('No consta')

# 2. TIPO VEHÍCULO 
mask_no_aplica = df_2023_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

# Peatón y testigo → No aplica 
df_2023_copy.loc[ mask_no_aplica & df_2023_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo' ] = 'No aplica'

# El resto de nulos → No consta 
df_2023_copy['tipo_vehiculo'] = df_2023_copy['tipo_vehiculo'].fillna('No consta') 

# 3. LESIVIDAD (ORDINAL) 
df_2023_copy['cod_lesividad'] = df_2023_copy['cod_lesividad'].fillna(14) 
df_2023_copy['lesividad'] = df_2023_copy['lesividad'].fillna('Sin asistencia sanitaria')

# 4. ALCOHOL Y DROGAS 
df_2023_copy['positiva_alcohol'] = df_2023_copy['positiva_alcohol'].fillna('No consta') 
df_2023_copy['positiva_droga'] = df_2023_copy['positiva_droga'].fillna('No consta')

#5. NUMERO 
df_2023_copy['numero'] = df_2023_copy['numero'].fillna('No aplica') 

- Vemos si los nulos de 'cod_distrito' y 'distrito' los podemos rellenar:

In [10]:
# Si me devuelve True es que los nulos de 'cod_distrito' son los mismos que los de 'distrito'
df_2023_copy[df_2023_copy["cod_distrito"].isna()].equals(df_2023_copy[df_2023_copy["distrito"].isna()])

True

In [11]:
df_2023_copy[df_2023_copy["distrito"].isna()]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
10110,2023s009762,2023-03-16,22:40:00,gta. isidro gonzalez velazquez / call. francis...,No aplica,NaN,NaN,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,446979.0,4481398.0,N,No consta,2023-03-16 22:40:00
38239,2023s032768,2023-10-21,00:40:00,villaverde a vallecas (farola 11),No aplica,NaN,NaN,Alcance,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,441291.0,4467125.0,N,No consta,2023-10-21 00:40:00
38240,2023s032768,2023-10-21,00:40:00,villaverde a vallecas (farola 11),No aplica,NaN,NaN,Alcance,Despejado,VMU eléctrico,Conductor,De 18 a 20 años,Hombre,3.0,Ingreso superior a 24 horas,441291.0,4467125.0,N,No consta,2023-10-21 00:40:00


Primero comprobamos que no haya más expedientes con esos números que sí tengan los valores de distrito y su código:

In [12]:
df_2023_copy[df_2023_copy["num_expediente"] == "2023s009762"]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
10110,2023s009762,2023-03-16,22:40:00,gta. isidro gonzalez velazquez / call. francis...,No aplica,NaN,NaN,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,446979.0,4481398.0,N,No consta,2023-03-16 22:40:00


In [13]:
df_2023_copy[df_2023_copy["num_expediente"] == "2023s032768"]

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
38239,2023s032768,2023-10-21,00:40:00,villaverde a vallecas (farola 11),No aplica,NaN,NaN,Alcance,Despejado,Turismo,Conductor,De 25 a 29 años,Hombre,14.0,Sin asistencia sanitaria,441291.0,4467125.0,N,No consta,2023-10-21 00:40:00
38240,2023s032768,2023-10-21,00:40:00,villaverde a vallecas (farola 11),No aplica,NaN,NaN,Alcance,Despejado,VMU eléctrico,Conductor,De 18 a 20 años,Hombre,3.0,Ingreso superior a 24 horas,441291.0,4467125.0,N,No consta,2023-10-21 00:40:00


Como no los hay la siguiente opción es buscar en Internet.
En el caso de la Glorieta, está en Hortaleza y buscando por coordenadas vemos que la Carretera de Villaverde a Vallecas (farola 11) está en Villaverde, así que buscamos cuáles son sus códigos de distrito:

In [21]:
df_2023_copy[df_2023_copy["distrito"] == "hortaleza".upper()].head(1)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
59,2023s000029,2023-01-01,10:20:00,plaza. alcalde moreno torres / avda pi y margal,1a,16.0,HORTALEZA,Choque contra obstáculo fijo,Nublado,Turismo,Conductor,Desconocido,Desconocido,14.0,Sin asistencia sanitaria,444340.0,4482134.0,N,No consta,2023-01-01 10:20:00


In [22]:
df_2023_copy[df_2023_copy["distrito"] == "villaverde".upper()].head(1)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
18,2023s000007,2023-01-01,05:35:00,"call. arroyo de la bulera, 11a",11a,17.0,VILLAVERDE,Otro,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,14.0,Sin asistencia sanitaria,442149.0,4465639.0,N,No consta,2023-01-01 05:35:00


In [23]:
# 6. DISTRITO Y SU CÓDIGO
mascara1 = df_2023_copy["distrito"].isna() & (df_2023_copy["num_expediente"] == "2023s009762")
mascara2 = df_2023_copy["cod_distrito"].isna() & (df_2023_copy["num_expediente"] == "2023s009762")
mascara3 = df_2023_copy["distrito"].isna() & (df_2023_copy["num_expediente"] == "2023s032768")
mascara4 = df_2023_copy["cod_distrito"].isna() & (df_2023_copy["num_expediente"] == "2023s032768")
df_2023_copy.loc[mascara1, "distrito"] = "hortaleza".upper()
df_2023_copy.loc[mascara2, "cod_distrito"] = float(16)
df_2023_copy.loc[mascara3, "distrito"] = "villaverde".upper()
df_2023_copy.loc[mascara4, "cod_distrito"] = float(17)

In [26]:
# 7. CONVERTIR A CATEGORY (DESPUÉS DE IMPUTAR) 
cols_category = [ 'estado_meteorológico', 
                 'tipo_vehiculo', 
                 'tipo_persona', 
                 'tipo_accidente', 
                 'distrito', 
                 'cod_distrito', 
                 'rango_edad', 
                 'sexo', 
                 'cod_lesividad', 
                 'lesividad', 
                 'positiva_alcohol', 
                 'positiva_droga' 
                 ] 
for col in cols_category: 
    df_2023_copy[col] = df_2023_copy[col].astype('category')

In [27]:
df_2023_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48830 entries, 0 to 48829
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   num_expediente        48830 non-null  string        
 1   fecha                 48830 non-null  datetime64[ns]
 2   hora                  48830 non-null  object        
 3   localizacion          48830 non-null  string        
 4   numero                48830 non-null  string        
 5   cod_distrito          48830 non-null  category      
 6   distrito              48830 non-null  category      
 7   tipo_accidente        48830 non-null  category      
 8   estado_meteorológico  48830 non-null  category      
 9   tipo_vehiculo         48830 non-null  category      
 10  tipo_persona          48830 non-null  category      
 11  rango_edad            48830 non-null  category      
 12  sexo                  48830 non-null  category      
 13  cod_lesividad   

# 2022

- Primero cargamos el dataset de acccidentalidad de Madrid del año 2022:

In [29]:
df_2022 = pd.read_csv(
    "./data/2022_Accidentalidad.csv",
    sep=";",
    skip_blank_lines=True,
    on_bad_lines="skip",
    encoding="utf-8"
)

Ahora tambien quiero ver sus dimensiones, es decir cuantas filas y columnas tiene el Dataset

In [30]:
df_2022.shape

(47053, 19)

- Aquí observamos que tiene 19 columnas igual que 2025, 2024 y 2023, y tiene 47053 filas

Ahora para el formato real de los datos y detectar valores a simple vista

In [31]:
df_2022

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,Conductor,De 30 a 34 años,Mujer,NaN,NaN,443359.0,4472082.0,N,NaN
1,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,NaN,NaN,443359.0,4472082.0,N,NaN
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,Conductor,De 30 a 34 años,Hombre,NaN,NaN,441155.0,4474129.0,S,NaN
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,Pasajero,De 35 a 39 años,Mujer,NaN,NaN,441155.0,4474129.0,N,NaN
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3,RETIRO,Colisión fronto-lateral,NaN,Turismo,Conductor,De 40 a 44 años,Hombre,NaN,NaN,441155.0,4474129.0,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47048,2022S041289,29/12/2022,19:40:00,CALL. GENERAL RICARDOS / plaza de toros,174,11,CARABANCHEL,Alcance,Despejado,Turismo,Conductor,De 50 a 54 años,Hombre,14.0,Sin asistencia sanitaria,437453.0,4470784.0,N,NaN
47049,2022S041289,29/12/2022,19:40:00,CALL. GENERAL RICARDOS / plaza de toros,174,11,CARABANCHEL,Alcance,Despejado,Turismo,Pasajero,De 50 a 54 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,437453.0,4470784.0,N,NaN
47050,2022S041312,12/12/2022,22:50:00,AVDA. ANDALUCIA / AVDA. PERLA,14,12,USERA,Colisión fronto-lateral,Lluvia débil,Turismo,Conductor,De 21 a 24 años,Mujer,NaN,NaN,441049.0,4469136.0,N,NaN
47051,2022S041312,12/12/2022,22:50:00,AVDA. ANDALUCIA / AVDA. PERLA,14,12,USERA,Colisión fronto-lateral,Lluvia débil,Turismo,Conductor,De 35 a 39 años,Hombre,NaN,NaN,441049.0,4469136.0,N,NaN


In [32]:
df_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47053 entries, 0 to 47052
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   num_expediente        47053 non-null  object 
 1   fecha                 47053 non-null  object 
 2   hora                  47053 non-null  object 
 3   localizacion          47053 non-null  object 
 4   numero                47053 non-null  object 
 5   cod_distrito          47053 non-null  int64  
 6   distrito              47053 non-null  object 
 7   tipo_accidente        47053 non-null  object 
 8   estado_meteorológico  41763 non-null  object 
 9   tipo_vehiculo         46854 non-null  object 
 10  tipo_persona          47053 non-null  object 
 11  rango_edad            47053 non-null  object 
 12  sexo                  47053 non-null  object 
 13  cod_lesividad         24905 non-null  float64
 14  lesividad             24905 non-null  object 
 15  coordenada_x_utm   

Aquí procedemos a hacer una copia del dataset para dejar el original tal cual y poder ver diferencias a la hora de limpiar y demás.

In [33]:
df_2022_copy = df_2022.copy()

Vamos a generar una tabla para ver los nulos y la cardinalidad de las diferentes variables que nos ayudaran a tomar decisiones con cada variable.

In [34]:
# Número de nulos 
nulos = df_2022_copy.isna().sum() 

# Porcentaje de nulos 
nulos_pct = (nulos / len(df_2022_copy) * 100).round(2) 

# Cardinalidad 
card = df_2022_copy.nunique() 

# Porcentaje de cardinalidad 
card_pct = (card / len(df_2022_copy) * 100).round(2) 

# Tipo de dato 
tipos = df_2022_copy.dtypes 

#Construcción de la tabla 
tabla_eda = pd.DataFrame({ 
    "nulos": nulos,
    "% nulos": nulos_pct,
    "cardinalidad": card, 
    "% cardinalidad": card_pct,
    "tipo": tipos }) 

# Ordenar por cardinalidad (o por nulos si prefieres) 
tabla_eda = tabla_eda.sort_values("cardinalidad", ascending=False) 
tabla_eda

,nulos,% nulos,cardinalidad,% cardinalidad,tipo
num_expediente,0,0.00,20212,42.96,object
localizacion,0,0.00,15146,32.19,object
coordenada_y_utm,10,0.02,10802,22.96,float64
coordenada_x_utm,10,0.02,10111,21.49,float64
hora,0,0.00,1207,2.57,object
numero,0,0.00,913,1.94,object
fecha,0,0.00,365,0.78,object
tipo_vehiculo,199,0.42,32,0.07,object
cod_distrito,0,0.00,21,0.04,int64
distrito,0,0.00,21,0.04,object


In [35]:
df_2022_copy["fecha"] = pd.to_datetime(df_2022_copy["fecha"], errors="coerce", dayfirst=True) 
df_2022_copy["hora"] = pd.to_datetime(df_2022_copy["hora"], format="%H:%M:%S", errors="coerce").dt.time 
df_2022_copy["fecha_hora"] = df_2022_copy.apply( 
                            lambda row: pd.Timestamp.combine(row["fecha"], row["hora"])
                            if pd.notnull(row["fecha"]) and pd.notnull(row["hora"])
                            else pd.NaT,
                            axis=1 )

cols_string = ["num_expediente", "localizacion", "numero"] 
for col in cols_string: 
    df_2022_copy[col] = df_2022_copy[col].astype("string").str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

- Ahora vamos a tratar los nulos 

In [37]:
# 1. ESTADO METEOROLÓGICO
df_2022_copy['estado_meteorológico'] = df_2022_copy['estado_meteorológico'].fillna('No consta')

# 2. TIPO VEHÍCULO 
mask_no_aplica = df_2022_copy['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 

# Peatón y testigo → No aplica 
df_2022_copy.loc[ mask_no_aplica & df_2022_copy['tipo_vehiculo'].isna(), 'tipo_vehiculo' ] = 'No aplica'

# El resto de nulos → No consta 
df_2022_copy['tipo_vehiculo'] = df_2022_copy['tipo_vehiculo'].fillna('No consta') 

# 3. LESIVIDAD (ORDINAL) 
df_2022_copy['cod_lesividad'] = df_2022_copy['cod_lesividad'].fillna(14) 
df_2022_copy['lesividad'] = df_2022_copy['lesividad'].fillna('Sin asistencia sanitaria')

# 4. ALCOHOL Y DROGAS 
df_2022_copy['positiva_alcohol'] = df_2022_copy['positiva_alcohol'].fillna('No consta') 
df_2022_copy['positiva_droga'] = df_2022_copy['positiva_droga'].fillna('No consta')

#5. NUMERO 
df_2022_copy['numero'] = df_2022_copy['numero'].fillna('No aplica') 

In [38]:
# 6. CONVERTIR A CATEGORY (DESPUÉS DE IMPUTAR) 
cols_category = [ 'estado_meteorológico', 
                 'tipo_vehiculo', 
                 'tipo_persona', 
                 'tipo_accidente', 
                 'distrito', 
                 'cod_distrito', 
                 'rango_edad', 
                 'sexo', 
                 'cod_lesividad', 
                 'lesividad', 
                 'positiva_alcohol', 
                 'positiva_droga' 
                 ] 
for col in cols_category: 
    df_2022_copy[col] = df_2022_copy[col].astype('category')

In [39]:
df_2022_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47053 entries, 0 to 47052
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   num_expediente        47053 non-null  string        
 1   fecha                 47053 non-null  datetime64[ns]
 2   hora                  47053 non-null  object        
 3   localizacion          47053 non-null  string        
 4   numero                47053 non-null  string        
 5   cod_distrito          47053 non-null  category      
 6   distrito              47053 non-null  category      
 7   tipo_accidente        47053 non-null  category      
 8   estado_meteorológico  47053 non-null  category      
 9   tipo_vehiculo         47053 non-null  category      
 10  tipo_persona          47053 non-null  category      
 11  rango_edad            47053 non-null  category      
 12  sexo                  47053 non-null  category      
 13  cod_lesividad   